### Library imports

In [1]:
import sys
print(sys.executable)

/dlabscratch1/berquet/conda/envs/env2/bin/python


In [2]:
import numpy
import torch
import transformers

/dlabscratch1/berquet/conda/envs/env2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

In [4]:
from trl import SFTTrainer

In [5]:
torch.cuda.get_device_name(0)

'NVIDIA A100-SXM4-40GB'

### Data imports

In [6]:
dataset = load_dataset("berquetR/dlab_project_optimal_links")


In [7]:
train_dataset = dataset['train']
validation_dataset = dataset['validation']

In [8]:
train_dataset

Dataset({
    features: ['source', 'target', 'current_page', 'current_page_links', 'next_page', '__index_level_0__'],
    num_rows: 78088
})

In [9]:
validation_dataset

Dataset({
    features: ['source', 'target', 'current_page', 'current_page_links', 'next_page', '__index_level_0__'],
    num_rows: 26178
})

### Build Prompt Version

In [10]:
def transform_instruction_format(examples):
    instructions = [
        f"Given two Wikipedia pages, a source page : {current_page} and a target page : {target}, your goal is to reach the target page starting from the source page in as few clicks as possible."
        for current_page, source, target in zip(examples['current_page'], examples['source'], examples['target'])
    ]
    inputs = examples['current_page_links']
    outputs = examples['next_page']

    return {'instruction': instructions, 'input': inputs, 'output': outputs}

In [11]:
# Before building prompt, build instruction, input, output format for the alpaca format
columns_to_remove = ['source', 'target', 'current_page', 'current_page_links', 'next_page','__index_level_0__' ]
train_dataset = train_dataset.map(transform_instruction_format, batched=True, remove_columns=columns_to_remove)
validation_dataset = validation_dataset.map(transform_instruction_format, batched=True, remove_columns=columns_to_remove)

In [12]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{} 


### Constraint:
You are provided the links available on the source page in the Input and should choose exclusively one link among those proposed links."


### Input:
{}


### Response:
{}"""

In [13]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) 
        texts.append(text)
    return { "text" : texts,}
pass

In [14]:
columns_to_remove_2 = ['instruction', 'input', 'output']
train_dataset=train_dataset.map(formatting_prompts_func, batched = True, remove_columns= columns_to_remove_2)
validation_dataset=validation_dataset.map(formatting_prompts_func, batched = True, remove_columns= columns_to_remove_2)

In [15]:
print(train_dataset[0])

{'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven two Wikipedia pages, a source page : Nationalism and a target page : Antwerp, your goal is to reach the target page starting from the source page in as few clicks as possible. \n\n\n### Constraint:\nYou are provided the links available on the source page in the Input and should choose exclusively one link among those proposed links."\n\n\n### Input:\n[\'Human\', \'Politics\', \'History of the world\', \'Denmark\', \'Culture\', \'Music\', \'Literature\', \'Folklore\', \'Mythology\', \'Religion\', \'Cultural identity\', \'Language\', \'Culture\', \'Ethnic group\', \'Gender\', \'Education\', \'Ottoman Empire\', \'Russia\', \'China\', \'Vatican City\', \'Roman Catholic Church\', \'Ethics\', \'War\', \'Marxism\', \'Socialism\', \'Sociology\', \'World War II\', \'French Revolution\', \'Napoleon\', \'

### Accelerator

In [16]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

### Model import 

In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [18]:
base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
model_mistral = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

tokenizer_mistral = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path = 'mistralai/Mistral-7B-Instruct-v0.2',
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer_mistral.pad_token = tokenizer.eos_token
tokenizer_mistral.padding_side = 'right'

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:59<00:00, 19.76s/it]


In [20]:
base__smaller_model_id = "microsoft/phi-1_5"
small_model = AutoModelForCausalLM.from_pretrained(base__smaller_model_id, quantization_config=bnb_config, device_map="auto")

In [21]:
small_tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path = "microsoft/phi-1_5",
    add_eos_token=True,
    add_bos_token=True,
    use_fast=False
)
small_tokenizer.pad_token = small_tokenizer.eos_token
small_tokenizer.padding_side = 'right'

### Lora

In [22]:
from peft import prepare_model_for_kbit_training

small_model.gradient_checkpointing_enable()
small_model = prepare_model_for_kbit_training(small_model)

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [23]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

#model = get_peft_model(model, config)
small_model = get_peft_model(small_model, config)

In [22]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    print('parallelizable')
    model.is_parallelizable = True
    model.model_parallel = True

In [23]:
model = accelerator.prepare_model(model)

In [24]:
small_model = accelerator.prepare_model(small_model)

In [25]:
output_dir = './mistral-first_run_dlab'

In [26]:
training_arguments = TrainingArguments(
        output_dir = output_dir, 
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 1,
        
        warmup_steps = 5,
        learning_rate = 2e-5,
    
        #Keep precision at 32bit for training
        bf16 = True,
        fp16 = False, 
        #Perform eval every save_steps
        do_eval = True,
        #Saving based on number of steps
        save_strategy = 'steps',
        #Number of steps model checkpoints will be saved and evaluated
        save_steps = 500,
        eval_steps = 500,
        #Start reporting loss
        logging_steps = 1,
        #Use paged optimizer for memory efficiency
        optim = "paged_adamw_8bit", 
        #2 epochs
        num_train_epochs = 2, 
    )

In [84]:
training_arguments_test = TrainingArguments(
        output_dir = output_dir, 
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1,
        
        warmup_steps = 5,
        learning_rate = 2e-5,
    
        #Keep precision at 32bit for training
        bf16 = True,
        #Perform eval every save_steps
        do_eval = True,
        #Saving based on number of steps
        save_strategy = 'steps',
        evaluation_strategy = 'steps',
        #Number of steps model checkpoints will be saved and evaluated
        save_steps = 10,
        eval_steps = 10,
        max_steps = 30,
        #Start reporting loss
        logging_steps = 1,
        #Use paged optimizer for memory efficiency
        optim = "paged_adamw_8bit", 
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
trainer = SFTTrainer(
    model = small_model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset= validation_dataset,
    dataset_text_field = "text",
    max_seq_length = 1024,
    args = training_arguments,
    peft_config = config,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26178/26178 [00:12<00:00, 2132.74 examples/s]
/dlabscratch1/berquet/conda/envs/env2/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [35]:
trainer.train()

/dlabscratch1/berquet/conda/envs/env2/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.861200
2,1.636600


KeyboardInterrupt: 

In [27]:
trainer_small = SFTTrainer(
    model = small_model,
    tokenizer = small_tokenizer,
    train_dataset = train_dataset,
    eval_dataset= validation_dataset,
    dataset_text_field = "text",
    max_seq_length = 1024,
    args = training_arguments,
    peft_config = config,
    data_collator=transformers.DataCollatorForLanguageModeling(small_tokenizer, mlm=False),
)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26178/26178 [01:37<00:00, 268.42 examples/s]
/dlabscratch1/berquet/conda/envs/env2/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [28]:
trainer_small.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/dlabscratch1/berquet/conda/envs/env2/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.116000
2,1.892300


KeyboardInterrupt: 

### Saving the model

In [47]:
model.save_pretrained('mistral-first_run_dlab')

/dlabscratch1/berquet/conda/envs/env2/lib/python3.8/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


### Load Test Dataset

In [54]:
test_dataset = dataset['test']

In [55]:
test_dataset = test_dataset.map(transform_instruction_format, batched=True, remove_columns=columns_to_remove)

In [56]:
test_dataset[0]

{'instruction': 'Given two Wikipedia pages, a source page : History_of_computing_hardware and a target page : I_Want_to_Hold_Your_Hand, your goal is to reach the target page starting from the source page in as few clicks as possible.',
 'input': "['History of the Internet', 'Clay', 'Abacus', 'Computers', 'Abacus', 'China', 'Abacus', 'Abacus', 'Johannes Kepler', 'Blaise Pascal', 'Gottfried Leibniz', 'Leibniz', 'Charles Babbage', 'Moon', 'Charles Babbage', 'United States Constitution', 'World War II', 'Computer programming', 'Charles Babbage', 'Richard Feynman', 'Integrated circuit', 'World War II', 'Computer programming', 'World War II', 'Electronics', 'Mercury (element)', 'Alan Turing', 'Alan Turing', 'Computer science', 'Algorithm', 'Germany', 'Charles Babbage', 'Programming language', 'World War II', 'World War II', 'World War II', 'Alan Turing', 'Boolean logic', 'John von Neumann', 'Electronics', 'University of Cambridge', 'Soviet Union', 'Ukraine', 'Programming language', 'Central 

### Reload Model

In [51]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.94s/it]


In [52]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "mistral-first_run_dlab")

In [60]:
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

### Test Model

In [63]:
model_preds = []
for i in range(20) :
    input = eval_tokenizer(
    [
      alpaca_prompt.format(
        test_dataset[i]['instruction'], # instruction
          test_dataset[i]['input'], # input
          "", # output - leave this blank for generation!
      )
    ], return_tensors = "pt").to("cuda")
    outputs = ft_model.generate(**input, max_new_tokens = 64, use_cache = True)
    decoded_output = tokenizer.batch_decode(outputs)
    print(decoded_output)
    final_output = retrieve_output(decoded_output)
    model_preds.append(final_output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven two Wikipedia pages, a source page : History_of_computing_hardware and a target page : I_Want_to_Hold_Your_Hand, your goal is to reach the target page starting from the source page in as few clicks as possible. \n\n\n### Constraint:\nYou are provided the links available on the source page in the Input and should choose exclusively one link among those proposed links."\n\n\n### Input:\n[\'History of the Internet\', \'Clay\', \'Abacus\', \'Computers\', \'Abacus\', \'China\', \'Abacus\', \'Abacus\', \'Johannes Kepler\', \'Blaise Pascal\', \'Gottfried Leibniz\', \'Leibniz\', \'Charles Babbage\', \'Moon\', \'Charles Babbage\', \'United States Constitution\', \'World War II\', \'Computer programming\', \'Charles Babbage\', \'Richard Feynman\', \'Integrated circuit\', \'World War II\', \'Computer programm

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven two Wikipedia pages, a source page : World_War_II and a target page : I_Want_to_Hold_Your_Hand, your goal is to reach the target page starting from the source page in as few clicks as possible. \n\n\n### Constraint:\nYou are provided the links available on the source page in the Input and should choose exclusively one link among those proposed links."\n\n\n### Input:\n[\'Battle of Normandy\', \'Nuclear weapon\', \'United States\', \'Soviet Union\', \'Europe\', \'Cold War\', \'United Kingdom\', \'United States\', \'Nazi Germany\', \'Joseph Stalin\', \'Winston Churchill\', \'Adolf Hitler\', \'Benito Mussolini\', \'War\', \'Adolf Hitler\', \'Nazi Germany\', \'Poland\', \'Soviet Union\', \'North Africa\', \'Royal Air Force\', \'Royal Navy\', \'Soviet Union\', \'Battle of Stalingrad\', \'Battle of Norma

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven two Wikipedia pages, a source page : Phase_(matter) and a target page : Philosophy, your goal is to reach the target page starting from the source page in as few clicks as possible. \n\n\n### Constraint:\nYou are provided the links available on the source page in the Input and should choose exclusively one link among those proposed links."\n\n\n### Input:\n[\'Physical science\', \'Set\', \'Gas\', \'Plasma (physics)\', \'Liquid crystal\', \'Magnet\', \'Thermodynamics\', \'Thermodynamics\', \'Entropy\', \'Entropy\', \'Glass\', \'Crystal\', \'Liquid crystal\', \'Ice\', \'Diamond\', \'Carbon\', \'Electron\', \'Plasma (physics)\', \'Absolute zero\', \'Metal\', \'Semiconductor\', \'Star\', \'Big Bang\', \'Photon\', \'Water\', \'Diamond\', \'Carbon\', \'Steel\', \'Iron\', \'Ice\']\n\n\n### Response:\nThe 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven two Wikipedia pages, a source page : Physical_science and a target page : Philosophy, your goal is to reach the target page starting from the source page in as few clicks as possible. \n\n\n### Constraint:\nYou are provided the links available on the source page in the Input and should choose exclusively one link among those proposed links."\n\n\n### Input:\n[\'Science\', \'Biology\', \'Astronomy\', \'Chemistry\', \'Geology\', \'Hydrology\', \'Meteorology\', \'Oceanography\', \'Soil science\', \'Physics\', \'Science\', \'Astronomy\', \'Science\', \'Stars\', \'Galaxy\', \'Big Bang\', \'Universe\', \'Nicolaus Copernicus\', \'Heliocentrism\', \'Solar system\', \'Asteroid\', \'Earth\', \'Moon\', \'Astrology\', \'Chemistry\', \'Science\', \'Matter\', \'Quantum mechanics\', \'Chemical element\', \'Dmitri

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven two Wikipedia pages, a source page : Science and a target page : Philosophy, your goal is to reach the target page starting from the source page in as few clicks as possible. \n\n\n### Constraint:\nYou are provided the links available on the source page in the Input and should choose exclusively one link among those proposed links."\n\n\n### Input:\n[\'Society\', \'Mathematics\', \'Atom\', \'History of science\', \'Latin\', \'Sanskrit\', \'Middle Ages\', \'Age of Enlightenment\', \'Philosophy\', \'Physical science\', \'Ethics\', \'Nature\', \'Universe\', \'Quantum mechanics\', \'Mathematics\', \'Calculus\', \'Statistics\', \'Topology\', \'Physics\', \'Chemistry\', \'Richard Feynman\', \'Bertrand Russell\', \'John Locke\', \'Chemistry\', \'Biology\', \'Ethics\', \'Pollution\', \'University\', \'Tech

### Import Data links

In [45]:
page_name_links = load_dataset("berquetR/file_names_links")

### Testing Agent

In [117]:
def links_from_source(dataset, source):
    filtered = page_name_links['train'].filter(lambda example : example['filename'] == source)

    if len(filtered) > 0:
        row_with_filename_e = filtered[0]
        links = row_with_filename_e['links']
        return links
    else:
        print("No rows found where filename = 'e'")

In [58]:
import re
def retrieve_output (model_output) :
    match = re.search(r"### Response:\n(.+?)</s>", model_output[0])

    if match:
        response_value = str(match.group(1).lstrip())
        return response_value
    else:
        print("The value could not be extracted.")

In [119]:
def get_prompt(source_links, source, original_source, target):
    instruction = f"We are currently on the Wikipedia page: {source}, we originally started on the Wikipedia page: {original_source}, given the following Wikipedia pages links, on which one should we click on to arrive the fastest to the Wikipedia page in terms of visited pages : {target}."

    input = source_links
    output = ' '

    return {'instruction': instruction, 'input': input, 'output': output}

In [141]:
model_choice = 'Nationalism'
original_source = 'Nationalism'
source_links = '[\'Human\', \'Politics\', \'History of the world\', \'Denmark\', \'Culture\', \'Music\', \'Literature\', \'Folklore\', \'Mythology\', \'Religion\', \'Cultural identity\', \'Language\', \'Culture\', \'Ethnic group\', \'Gender\', \'Education\', \'Ottoman Empire\', \'Russia\', \'China\', \'Vatican City\', \'Roman Catholic Church\', \'Ethics\', \'War\', \'Marxism\', \'Socialism\', \'Sociology\', \'World War II\', \'French Revolution\', \'Napoleon\', \'Capitalism\', \'Mythology\', \'French Revolution\', \'Liberalism\', \'Ottoman Empire\', \'Spain\', \'South America\', \'Netherlands\', \'England\', \'Asia\', \'India\', \'Mahatma Gandhi\', \'China\', \'Japan\', \'World War I\', \'Ottoman Empire\', \'Woodrow Wilson\', \'Iraq\', \'Lebanon\', \'Syria\', \'Russian Revolution of 1917\', \'Ireland\', \'Northern Ireland\', \'Fascism\', \'Nazism\', \'Italy\', \'Germany\', \'World War II\', \'The Holocaust\', \'Africa\', \'Somalia\', \'Portugal\', \'Mozambique\', \'Angola\', \'Belarus\', \'Ukraine\', \'Moldova\', \'Kazakhstan\', \'Turkmenistan\', \'Uzbekistan\', \'Tajikistan\', \'Kyrgyzstan\', \'Armenia\', \'Azerbaijan\', \'Georgia (country)\', \'Latvia\', \'Estonia\', \'Lithuania\', \'Yugoslavia\', \'Northern Ireland\', \'European Union\', \'Globalization\', "People\'s Republic of China", \'European Union\', \'French Revolution\', \'19th century\', \'Literacy\', \'Newspaper\', \'Book\', \'Ireland\', \'India\', \'Hebrew language\', \'United States\', \'German language\', \'World War I\', \'French language\', \'French language\', \'Spanish language\', \'English language\', \'Arabic language\', \'Algeria\', \'Western Sahara\', \'Nobiin language\', \'Egypt\', \'Sudan\', \'Morocco\', \'Civil society\', \'Citizenship\', \'Jean-Jacques Rousseau\', \'Liberalism\', \'Race\', \'Romanticism\', \'Folklore\', \'Brothers Grimm\', \'Culture\', \'Taiwan\', \'Fascism\', \'Liberalism\', \'Religion\', \'Ireland\', \'18th century\', \'Northern Ireland\', \'India\', \'Hinduism\', \'Zionism\', \'World War I\', \'Treaty of Versailles\', \'Lithuania\', \'Fascism\', \'Socialism\', \'Nazism\', \'Adolf Hitler\', \'Nazi Germany\', \'Nazism\', \'Race\', \'Nazi Germany\', \'Ottoman Empire\', \'Liberalism\', \'War\', \'Germany\', \'Marxism\', \'World War I\', \'Politics\', \'Anarchism\', \'Race\', \'Islam\', \'Government\', \'Islam\', \'United Nations\']'
target = 'Antwerp'

In [142]:
while target != model_choice :

    source = model_choice
    source_links = links_from_source(page_name_links , source)
    prompt =  get_prompt(source_links, source, original_source, target)
    input = eval_tokenizer(
    [
      alpaca_prompt.format(
        prompt['instruction'], # instruction
          prompt['input'], # input
          "", # output - leave this blank for generation!
      )
    ], return_tensors = "pt").to("cuda")
    outputs = ft_model.generate(**input, max_new_tokens = 64, use_cache = True)
    decoded_output = eval_tokenizer.batch_decode(outputs)
    print(decoded_output)
    model_choice = retrieve_output(decoded_output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWe are currently on the Wikipedia page: Nationalism, we originally started on the Wikipedia page: Nationalism, given the following Wikipedia pages links, on which one should we click on to arrive the fastest to the Wikipedia page in terms of visited pages : Antwerp.\n\n### Input:\n[\'Human\', \'Politics\', \'History of the world\', \'Denmark\', \'Culture\', \'Music\', \'Literature\', \'Folklore\', \'Mythology\', \'Religion\', \'Cultural identity\', \'Language\', \'Culture\', \'Ethnic group\', \'Gender\', \'Education\', \'Ottoman Empire\', \'Russia\', \'China\', \'Vatican City\', \'Roman Catholic Church\', \'Ethics\', \'War\', \'Marxism\', \'Socialism\', \'Sociology\', \'World War II\', \'French Revolution\', \'Napoleon\', \'Capitalism\', \'Mythology\', \'French Revolution\', \'Liberalism\', \'Ottoman Emp

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


["<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWe are currently on the Wikipedia page: European_Union, we originally started on the Wikipedia page: Nationalism, given the following Wikipedia pages links, on which one should we click on to arrive the fastest to the Wikipedia page in terms of visited pages : Antwerp.\n\n### Input:\n['Brussels', 'Dutch language', 'English language', 'French language', 'German language', 'Portuguese language', 'Spanish language', 'Austria', 'Belgium', 'Bulgaria', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Republic of Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'United Kingdom', 'Government', 'European Parliament', 'Germany', 'Water', 'Currency', 'Curr

In [139]:
e = dataset['train']

In [140]:
e[0]

{'source': 'Nationalism',
 'target': 'Antwerp',
 'current_page': 'Nationalism',
 'current_page_links': '[\'Human\', \'Politics\', \'History of the world\', \'Denmark\', \'Culture\', \'Music\', \'Literature\', \'Folklore\', \'Mythology\', \'Religion\', \'Cultural identity\', \'Language\', \'Culture\', \'Ethnic group\', \'Gender\', \'Education\', \'Ottoman Empire\', \'Russia\', \'China\', \'Vatican City\', \'Roman Catholic Church\', \'Ethics\', \'War\', \'Marxism\', \'Socialism\', \'Sociology\', \'World War II\', \'French Revolution\', \'Napoleon\', \'Capitalism\', \'Mythology\', \'French Revolution\', \'Liberalism\', \'Ottoman Empire\', \'Spain\', \'South America\', \'Netherlands\', \'England\', \'Asia\', \'India\', \'Mahatma Gandhi\', \'China\', \'Japan\', \'World War I\', \'Ottoman Empire\', \'Woodrow Wilson\', \'Iraq\', \'Lebanon\', \'Syria\', \'Russian Revolution of 1917\', \'Ireland\', \'Northern Ireland\', \'Fascism\', \'Nazism\', \'Italy\', \'Germany\', \'World War II\', \'The Ho